## Note: run only if needed!

In [20]:
import pandas as pd
import os

In [51]:
def move_labels_and_remove_missing_files(file_path, labels):
    files = [f for f in os.listdir(file_path) if f.endswith('jpg')]
    for f in files:
        if f not in labels.index:
            if os.path.exists(os.path.join(file_path, f)):   # shouldn't need to do this, but better to be safe
                os.remove(os.path.join(file_path, f))
        prefix = f.split('_jpg')[0]
        num = ''
        for char in reversed(prefix):
            if char.isdigit():
                num = char + num
            else:
                break
        padded_filename = num.zfill(4) + ".jpg"
        if os.path.exists(os.path.join(file_path, f)):
            os.rename(os.path.join(file_path, f), os.path.join(file_path, padded_filename))
        labels = labels.rename(index={f: padded_filename})

    return labels

In [56]:
def create_label_file(file_path, orig_df):
    labels = orig_df.copy(deep=True)
    labels['visibility'] = 1
    
    x_coord = (labels.loc[:, 'xmin'] + labels.loc[:, 'xmax']) // 2
    labels['x-coordinate'] = x_coord

    y_coord = (labels.loc[:, 'ymin'] + labels.loc[:, 'ymax']) // 2
    labels['y-coordinate'] = y_coord

    labels['status'] = 0

    labels.index.names = ['file name']
    labels = labels.drop(['width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax'], axis=1)
    labels = labels.sort_index(ascending=True)

    labels.to_csv(file_path + "/Label.csv", index=True)

In [31]:
label = pd.read_csv("./basketball_dataset/train/_annotations.csv").set_index('filename')
label = label.sort_values(by=['filename'])

In [38]:
davis_labels = label[label.index.str.startswith('davis')]
high2lights_labels = label[label.index.str.startswith('high2lights')]
highlights_labels = label[label.index.str.startswith('highlights')]
madeshot_labels = label[label.index.str.startswith('madeshot')]

97


In [57]:
davis_filepath = "./basketball_dataset/davis"

davis_labels = move_labels_and_remove_missing_files(davis_filepath, davis_labels)
create_label_file(davis_filepath, davis_labels)

In [58]:
high2lights_filepath = "./basketball_dataset/high2lights"

high2lights_labels = move_labels_and_remove_missing_files(high2lights_filepath, high2lights_labels)
create_label_file(high2lights_filepath, high2lights_labels)

In [59]:
highlights_filepath = "./basketball_dataset/highlights"

highlights_labels = move_labels_and_remove_missing_files(highlights_filepath, highlights_labels)
create_label_file(highlights_filepath, highlights_labels)

In [60]:
madeshot_filepath = "./basketball_dataset/madeshot"

madeshot_labels = move_labels_and_remove_missing_files(madeshot_filepath, madeshot_labels)
create_label_file(madeshot_filepath, madeshot_labels)